<a href="https://colab.research.google.com/github/yiranlus/Ankia-Theme/blob/main/12.%20Machine%20Learning/ML1%20-%20Intro/MachineLearningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Project on Arbres

In [ ]:
# @title Download the Arbres Dataset
!curl -L "https://data.metropolegrenoble.fr/d4c/api/records/2.0/downloadfile/format=csv&resource_id=17fff084-3e54-4add-8d00-e2d31676fdec&use_labels_for_header=true&user_defined_fields=true" -o arbres_grenoble_epsg4326.csv


In [ ]:
# @title Import the Essential Modules
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import folium
from folium.plugins import FastMarkerCluster

try:
    from google.colab import data_table

    # Enable interactive table display
    data_table.enable_dataframe_formatter()
    from google.colab.data_table import DataTable
except:
    def DataTable(df, *args, **kwargs):
        display(df)


## Load and Clean the Dataset

In [ ]:
df = pd.read_csv("./arbres_grenoble_epsg4326.csv")
df.set_index("elem_point_id", inplace=True)
df["anneedeplantation"] = df["anneedeplantation"].astype(pd.Int64Dtype())
df_latlng = df["geo_point_2d"].str.split(",")
df["latitude"] = df_latlng.apply(lambda x: x[0])
df["longitude"] = df_latlng.apply(lambda x: x[1])
df.drop(columns="geo_point_2d", inplace=True)
DataTable(df.head(), max_columns=len(df.columns))

In [ ]:
# @title Remove Trees where "Année de Plantation" is missing

df_clean = df.dropna(subset=["anneedeplantation"])

In [ ]:
# @title Remove Unhelpful Columns

## All the rows have the same value
df_clean = df_clean.drop(columns=["genre", "genre_desc", "categorie", "categorie_desc"])

## All the rows have missing value
df_clean = df_clean.drop(columns=["equipe", "courrier", "anneeabattage", "essouchement", "diametrearbre", "causeabattage"])

DataTable(df_clean.head(), max_columns=len(df_clean.columns))


In [ ]:
df_clean.to_pickle("arbres_grenoble_epsg4326_clean.pickle")

print("saved to arbres_grenoble_epsg4326_clean.pickle")

In [ ]:
df_clean.typenature.unique()